In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import numpy.typing as npt
import torch as th

In [2]:
wavelengths1 = np.arange(390, 701, 1)
wavelengths10 = np.arange(400, 701, 10)

In [3]:
from chromalab.observer import Observer, Cone
from chromalab.inks import Neugebauer, InkGamut, CellNeugebauer, Pigment
from chromalab.spectra import Spectra, Illuminant

In [4]:
from tqdm.notebook import tqdm

## Load inks

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
cijk = {}
with open(f'../data/nix/010624.csv') as csvf:
    spamreader = csv.reader(csvf, delimiter=';')
    for i, row in enumerate(spamreader):
        if i < 4: continue
        if row[2] != "CIJK": continue
            
        name = row[4]
        color_data = np.array(row[33:],dtype=float)
        spectra = Spectra(data=color_data, wavelengths=wavelengths10)

        cijk[name] = spectra

In [7]:
d65 = Illuminant.get("D65")

In [8]:
tetrachromat_d65 = Observer.tetrachromat(illuminant=d65)

In [9]:
all_inks = {}
with open(f'../data/nix/02022024.csv') as csvf:
    spamreader = csv.reader(csvf, delimiter=';')
    for i, row in enumerate(spamreader):
        if i < 4: continue            
        name = row[4]
        color_data = np.array(row[33:],dtype=float)
        spectra = Spectra(data=color_data, wavelengths=wavelengths10)

        all_inks[name] = spectra
        
with open(f'../data/nix/011624.csv') as csvf:
    spamreader = csv.reader(csvf, delimiter=';')
    for i, row in enumerate(spamreader):
        if i < 4: continue            
        name = row[4]
        color_data = np.array(row[33:],dtype=float)
        try:
            spectra = Spectra(data=color_data, wavelengths=wavelengths10)
        except ValueError:
            continue

        all_inks[name] = spectra
        
        
with open(f'../data/nix/Inks_all.csv') as csvf:
    spamreader = csv.reader(csvf, delimiter=';')
    for i, row in enumerate(spamreader):
        if i < 4: continue            
        name = row[4]
        color_data = np.array(row[33:],dtype=float)
        spectra = Spectra(data=color_data, wavelengths=wavelengths10)

        all_inks[name] = spectra

/Users/varun/Documents/Python/ChromaLab/chromalab/spectra.py:35: UserWarning: Data has values not between 0 and 1. Clipping.
  warnings.warn("Data has values not between 0 and 1. Clipping.")


In [10]:
all_inks["Epson C"] = cijk["2000"]

In [11]:
paper = cijk["0000"]

In [12]:
len(all_inks)

149

## previous q spread

In [13]:
cijk_list = [all_inks["Epson C"], all_inks["Peach haze D"],  all_inks["Platinum violet"], all_inks["Operation overlord orange N"]]
cijk_gamut = InkGamut(cijk_list, paper=paper, illuminant=d65)

In [44]:
from sklearn.decomposition import PCA
pca3 = PCA(n_components=3)
pca4 = PCA(n_components=4)

In [40]:
cijk_points, _ =cijk_gamut.get_point_cloud(tetrachromat_d65)

Generating point cloud: 1it [00:00, 25.09it/s]


In [45]:
pca3.fit(cijk_points)
np.sqrt(pca3.explained_variance_)

array([ 0.10494273,  0.08536831,  0.01684877])

In [46]:
pca4.fit(cijk_points)
np.sqrt(pca4.explained_variance_)

array([ 0.10494273,  0.08536831,  0.01684877,  0.00094397])

In [31]:
m1 = cijk_gamut.get_spectra((0.05, 0, 0.85, 0.85))
m2 = cijk_gamut.get_spectra((0.4, 0.75, 0.05, 0.05))

In [35]:
tetrachromat_d65.observe(m1) 

array([ 0.28911962,  0.27740243,  0.3166211 ,  0.34557817])

In [38]:
point = tetrachromat_d65.observe(m1) 
tuple(int(100*round(p,2))for i, p in enumerate(point) if i != 3)

(28, 28, 32)

In [39]:
point = tetrachromat_d65.observe(m2) 
tuple(int(100*round(p,2))for i, p in enumerate(point) if i != 3)

(27, 28, 33)

In [36]:
tetrachromat_d65.observe(m2)

array([ 0.27474368,  0.29321296,  0.33350118,  0.35380439])

In [22]:
%time cijk_gamut.get_point_cloud(tetrachromat_d65, stepsize=0.05)[0].shape

Generating point cloud: 2it [00:00,  6.84it/s]                                                                                                                                                

CPU times: user 499 ms, sys: 1.12 s, total: 1.62 s
Wall time: 298 ms


(194481, 4)

In [21]:
cijk_gamut.get_width(tetrachromat_d65, stepsize=0.02)

Generating point cloud: 68it [00:11,  5.73it/s]                                                                                                                                               


Point cloud generated.
maximum distance is 0.021743220618915315 with percentages (0.68000000000000005, 0.040000000000000001, 0.64000000000000001, 0.44) and (0.64000000000000001, 0.46000000000000002, 0.14000000000000001, 0.02)


0.021743220618915315

In [15]:
cell_ijk = InkGamut(CellNeugebauer(cijk), illuminant=d65)
cell_ijk.get_width(tetrachromat_d65, stepsize=0.05)

Generating point cloud:   0%|                                                                                                                                     | 0/1.94481 [00:00<?, ?it/s]


TypeError: unhashable type: 'numpy.ndarray'

## Best Cyan

In [ ]:
pvy = [all_inks["Lamy red"], all_inks["Platinum violet"], all_inks["CT Pastel Yellow"]]

In [ ]:
gamut

In [ ]:
cyan_candidates = []
for name, spectra in tqdm(all_inks.items(), desc="Finding best cyan"):
    cpvy = pvy + [spectra]
    gamut = InkGamut(cpvy, paper=paper, illuminant=d65)
    width = gamut.get_width(tetrachromat_d65, verbose=False)
    cyan_candidates.append((width, name))

In [ ]:
cyan_candidates.sort(reverse=True)

In [ ]:
cyan_candidates

In [ ]:
all_inks["Vd parrot fish"].plot()
all_inks["R torquay"].plot()

In [ ]:
for width, name in cyan_candidates[:18]:
    cpvy = pvy + [all_inks[name]]
    gamut = InkGamut(cpvy, paper=paper, illuminant=d65)
    width = gamut.get_width(tetrachromat_d65, stepsize=0.05, verbose=False)
    print(name, width)

In [ ]:
# all_inks["Vd parrot fish"].plot()
all_inks["V isabela"].plot()
# all_inks["R torquay"].plot()
all_inks["Epson C"].plot(color='black')

## Best Pink

In [ ]:
all_inks["CT Pastel Yellow"]

In [ ]:
cvy = [all_inks["R torquay"], all_inks["Platinum violet"], all_inks["CT Pastel Yellow"]]

In [ ]:
pink_candidates = []
for name, spectra in tqdm(all_inks.items(), desc="Finding best pink"):
    cpvy = cvy + [spectra]
    gamut = InkGamut(cpvy, paper=paper, illuminant=d65)
    width = gamut.get_width(tetrachromat_d65, verbose=False)
    pink_candidates.append((width, name))

In [ ]:
pink_candidates.sort(reverse=True)

In [ ]:
pink_candidates

In [ ]:
for width, name in pink_candidates[:8]:
    cpvy = cvy + [all_inks[name]]
    gamut = InkGamut(cpvy, paper=paper, illuminant=d65)
    width = gamut.get_width(tetrachromat_d65, stepsize=0.05, verbose=False)
    print(name, width)

In [ ]:
all_inks["Peach haze D"].plot()
all_inks["Lamy red"].plot()

## Best Magenta

In [ ]:
cpy = [all_inks["R torquay"], all_inks["Lamy red"],  all_inks["CT Pastel Yellow"]]

In [ ]:
magenta_candidates = []
for name, spectra in tqdm(all_inks.items(), desc="Finding best magenta"):
    cpvy = cpy + [spectra]
    gamut = InkGamut(cpvy, paper=paper, illuminant=d65)
    width = gamut.get_width(tetrachromat_d65, verbose=False)
    magenta_candidates.append((width, name))

In [ ]:
magenta_candidates.sort(reverse=True)

In [ ]:
magenta_candidates

In [ ]:
all_inks["Platinum violet"].plot()
cijk["0020"].plot(color='black')
all_inks["Pink owl"].plot(color='green')

## Best Yellow

In [ ]:
cpv = [all_inks["R torquay"], all_inks["Lamy red"],all_inks["Platinum violet"]]

In [ ]:
yellow_candidates = []
for name, spectra in tqdm(all_inks.items(), desc="Finding best yellow"):
    cpvy = cpv + [spectra]
    gamut = InkGamut(cpvy, paper=paper, illuminant=d65)
    width = gamut.get_width(tetrachromat_d65, verbose=False)
    yellow_candidates.append((width, name))

In [ ]:
yellow_candidates.sort(reverse=True)

In [ ]:
yellow_candidates

In [ ]:
for width, name in yellow_candidates[:21]:
    cpvy = cpv + [all_inks[name]]
    gamut = InkGamut(cpvy, paper=paper, illuminant=d65)
    width = gamut.get_width(tetrachromat_d65, stepsize=0.05, verbose=False)
    print(name, width)

In [ ]:
all_inks["CT Pastel Yellow"].plot()
# all_inks["Rokyu moon yellow"].plot()

## Display best four inks

In [ ]:
all_inks["R torquay"].plot(name="R torquay")
all_inks["Lamy red"].plot(name="Lamy red")
all_inks["Platinum violet"].plot(name="Platinum violet")
all_inks["CT Pastel Yellow"].plot(name="CT Pastel Yellow")
plt.legend()
plt.show()

In [ ]:
best_cpvy = [all_inks["R torquay"], all_inks["Lamy red"],all_inks["Platinum violet"], all_inks["CT Pastel Yellow"]]

In [ ]:
best_cpvy_gamut = InkGamut(best_cpvy, paper=paper, illuminant=d65)

In [ ]:
best_cpvy_gamut.get_width(tetrachromat_d65, stepsize=0.05, save="cpvyFeb6")

## Best spot colors

In [ ]:
spot_candidates = []
for name, spectra in tqdm(all_inks.items(), desc="Finding best spot color"):
    cpvys = best_cpvy + [spectra]
    gamut = InkGamut(cpvys, paper=paper, illuminant=d65)
    width = gamut.get_width(tetrachromat_d65, stepsize=0.1, verbose=False)
    spot_candidates.append((width, name))

In [ ]:
spot_candidates.sort(reverse=True)

In [ ]:
spot_candidates

In [ ]:
best_spot_inks = spot_candidates[:17]

In [ ]:
all_inks["Purple jade rabbit"].plot()
all_inks["R honey bee"].plot()
all_inks["W dracula"].plot()
all_inks["FWP sparkling champagne"].plot()
all_inks["T copper patina"].plot()
all_inks["V maskara "].plot()


## Best two spot colors

In [ ]:
best_cpvy

In [ ]:
spot_candidates = []

# Iterate over all pairs of names and spectra in all_inks
for i, (_, name1) in enumerate(tqdm(best_spot_inks, desc="All pairs")):
    for (_, name2) in tqdm(best_spot_inks[i+1:]):
        # Create cpvys with the current pair of spectra added to best_cpvy
        cpvys = best_cpvy + [all_inks[name1], all_inks[name2]]
        gamut = InkGamut(cpvys, paper=paper, illuminant=d65)
        width = gamut.get_width(tetrachromat_d65, stepsize=0.2, verbose=False)
        
        # Append the result along with the names of the ink pair
        spot_candidates.append((width, name1, name2))

In [ ]:
spot_candidates.sort(reverse=True)

In [ ]:
spot_candidates

In [ ]:
for _, name1, name2 in spot_candidates[:10]:
    cpvyjk = best_cpvy + [all_inks[name1], all_inks[name2]]
    gamut = InkGamut(cpvyjk, paper=paper, illuminant=d65)
    width = gamut.get_width(tetrachromat_d65, stepsize=0.1, verbose=False)

    print(width, name1, name2)

In [ ]:
for _, name1, name2 in [spot_candidates[0], spot_candidates[0], spot_candidates[5], spot_candidates[8]]:
    cpvyjk = best_cpvy + [all_inks[name1], all_inks[name2]]
    gamut = InkGamut(cpvyjk, paper=paper, illuminant=d65)
    width = gamut.get_width(tetrachromat_d65, stepsize=0.05, verbose=True)

    print(width, name1, name2)